<a href="https://colab.research.google.com/github/SunkaraboinaPraveenKumar/Machine_Learning_Projects/blob/main/Logic_Puzzle_Solver_Deepseek.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch gradio pillow easyocr -q groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 76.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
import easyocr
import groq
reader = easyocr.Reader(['en'])

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

In [3]:
from google.colab import userdata
import os
os.environ['GROQ_API_KEY']=userdata.get('groq_api_key')

In [4]:
client = groq.Client(api_key=os.environ['GROQ_API_KEY'])

In [21]:
def solve_puzzle(image):
  try:
    image_path="uploaded_puzzle.png"
    image.save(image_path)

    results = reader.readtext(image_path)
    extracted_text = " ".join([res[1] for res in results])

    extracted_text.replace("??","?")
    if "?" not in extracted_text:
      extracted_text+="?"

    print("Extracted Text:",extracted_text)

    refined_text = extracted_text.replace('x','*').replace("X",'*').replace("="," = ").strip()
    print("Refined Text:",refined_text)
    puzzle_prompt = (
      f"You are an AI specialized in solving puzzles. Analyze the following, identify hidden patterns or rules, and provide the missing values step by step reasoning."
      f"\nPuzzle:\n {refined_text}\n"
      "Format Your Response strictly as follows:\n"
      "1. **Given Equation**:\n - (original equations)\n"
      "2. **Pattern Identified:**:\n (explain the hidden logic)\n"
      "3. **Step-by-step calculation**:\n - For (input values):\n  (calculation and result)\n"
      "4. **Final Answer**:\n - (Answer = X)\n"
      )

    response = client.chat.completions.create(
        model="deepseek-r1-distill-llama-70b",
        messages=[{"role":"user","content":puzzle_prompt}],
        temperature=0,
        max_tokens=1000
    )

    if response and response.choices:
      return response.choices[0].message.content.strip()
    else:
      return "Error"
  except Exception as e:
    return f"Error: {str(e)}"

In [22]:
!pip install pillow

In [23]:
from PIL import Image
img = Image.open("/content/puzzle.png")

In [24]:
solve_puzzle(img)

Extracted Text: 8 9 3 | 1| 8 | 9 9 | 7 | | C 2 | 8 | 6 | 5 | 2 1 | | 2 | 6 3 6 4 | 4 | 2 | 9 | 7 3 Ws] | 49 2 2 | | 3 | |5| 7 | 8 4 | 2 11 |5?
Refined Text: 8 9 3 | 1| 8 | 9 9 | 7 | | C 2 | 8 | 6 | 5 | 2 1 | | 2 | 6 3 6 4 | 4 | 2 | 9 | 7 3 Ws] | 49 2 2 | | 3 | |5| 7 | 8 4 | 2 11 |5?


"<think>\nOkay, so I'm trying to solve this puzzle that was given. It looks like a series of numbers and some missing values represented by question marks. The puzzle is laid out in a specific format, and I need to figure out the hidden pattern or rule that connects these numbers. Let me break it down step by step.\n\nFirst, I'll write down the given equation as it's presented:\n\n8 9 3 | 1| 8 | 9 9 | 7 | | C 2 | 8 | 6 | 5 | 2 1 | | 2 | 6 3 6 4 | 4 | 2 | 9 | 7 3 Ws] | 49 2 2 | | 3 | |5| 7 | 8 4 | 2 11 |5?\n\nHmm, this looks a bit messy. I notice that there are numbers separated by spaces and some are grouped with pipes |. There are also some letters like C and Ws, which might be abbreviations or part of the pattern. The question marks at the end indicate missing values that I need to find.\n\nI think the first step is to see if there's a mathematical relationship between the numbers. Maybe each group of numbers is connected through some operation like addition, subtraction, multiplicat

In [25]:
from PIL import Image
img = Image.open("/content/puzzle.jpg")

In [27]:
result = solve_puzzle(img)

Extracted Text: Can You Solve This Number Puzzle: ? 2 + 10 = 24 3 + 6 = 27 7 + 2 = 63 5 + 3 29 1
Refined Text: Can You Solve This Number Puzzle: ? 2 + 10  =  24 3 + 6  =  27 7 + 2  =  63 5 + 3 29 1


In [28]:
print(result)

<think>
Okay, so I've got this number puzzle here, and I need to figure out the missing value. Let me look at the equations given:

? 2 + 10 = 24  
3 + 6 = 27  
7 + 2 = 63  
5 + 3 = 29  
1

Hmm, I need to find out what the question mark is. Let me see if I can spot a pattern here.

Starting with the first equation: 2 + 10 equals 24. Wait, 2 plus 10 is 12, but the result is 24. That's double of 12. Maybe that's a clue. Let me check the next one.

3 + 6 equals 27. 3 plus 6 is 9, and 9 times 3 is 27. Oh, interesting. So maybe the operation isn't just addition. Let me see the next one.

7 + 2 equals 63. 7 plus 2 is 9, and 9 times 7 is 63. Wait, that's consistent with the previous one. So maybe the pattern is (a + b) multiplied by a? Let me test that.

For the first equation: (2 + 10) = 12, then 12 times 2 is 24. That works. Second equation: (3 + 6) = 9, 9 times 3 is 27. Third equation: (7 + 2) = 9, 9 times 7 is 63. Fourth equation: 5 + 3 = 8, 8 times 5 is 40, but the result is 29. Wait, th

In [29]:
from PIL import Image
img = Image.open("/content/puzzle.jpg")
result = solve_puzzle(img)
print(result)

Extracted Text: Salve the Nurnber Puzzle? 18 20 100 21 147 Sawaal.com
Refined Text: Salve the Nurnber Puzzle? 18 20 100 21 147 Sawaal.com
<think>
Okay, so I've got this puzzle here: 18, 20, 100, 21, 147, and I need to figure out the pattern or rule that connects these numbers. The question is asking for the missing value, which I assume is the next number in the sequence or maybe the one that replaces "Sawaal.com" if that's part of the puzzle. Let me start by writing down the numbers to see if I can spot any obvious patterns.

First, the numbers given are 18, 20, 100, 21, 147. Hmm, they don't seem to be in an arithmetic or geometric progression. Let me check the differences between consecutive numbers:

20 - 18 = 2
100 - 20 = 80
21 - 100 = -79
147 - 21 = 126

These differences don't form a clear pattern. Maybe it's something else. Let me think about other possibilities. Sometimes, in number puzzles, each number is derived from the previous one using a specific operation or rule. Maybe 

In [ ]:
import gradio as gr

interface = gr.Interface(
    fn=solve_puzzle,
    inputs=gr.Image(type='pil'),
    outputs="text",
    title="Logic Puzzle Solver",
    description="Upload an image of a logic puzzle and get the solution."
)
interface.launch(debug=True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://d7310e0034a9bf38d9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Extracted Text: Salve the Nurnber Puzzle? 18 20 100 21 147 Sawaal.com
Refined Text: Salve the Nurnber Puzzle? 18 20 100 21 147 Sawaal.com
